#### Import
- MTCNN : 얼굴 검출 후 얼굴 크롭하는 라이브러리

In [ ]:
import os
import cv2 as cv
import numpy as np
from facenet_pytorch import MTCNN
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"]= "1"

- REAL과 FAKE 폴더에 속한 모든 폴더 경로 리스트 생성

In [ ]:
real_dir = ('C:/ai/workspace/sh/REAL/')
real_list = [real_dir + x for x in os.listdir(real_dir)]
real_list = list(map(lambda x: x+"/", real_dir))

In [ ]:
fake_dir = ('C:/ai/workspace/sh/FAKE/')
fake_list = [fake_dir + x for x in os.listdir(fake_dir)]
fake_list = list(map(lambda x: x+"/", fake_list))

### Face Detect
- v_cap : 영상을 프레임별로 자름
- v_len : 프레임의 총 개수를 변수에 저장
- cv2.VideoCapture.grab() : 프레임을 가져온다. 실패 시 success 변수엔 False가 저장
- cv2.VideoCapture.retrieve() : grab한 프레임을 decode한다. 프레임이 없을 경우 False가 success 변수에 저장
- cv.COLOR_BGR2RGB : 프레임을 BGR에서 RGB로 변환
- MTCNN : 얼굴을 검출하고, margin을 줘 얼굴이 살짝 잘리는 것을 방지
- save_path : 캡쳐 파일 저장
- count : 영상에서 프레임별로 검출되는 모든 얼굴에 대해 번호를 매기기 위함

In [ ]:
def chimac(videos, num, label):
    
    count=0
    file =  videos.split("/")[-1]
    fileName = file.split(".")[0]
    mtcnn = MTCNN(margin=50, keep_all=True, post_process=True, device='cuda:0',thresholds=[.9,.9,.9])
    v_cap = cv.VideoCapture(videos)
    v_len = int(v_cap.get(cv.CAP_PROP_FRAME_COUNT))
    
    for i in tqdm(range(v_len)):    
        success = v_cap.grab()
        if i % 1 == 0:
            success, frame = v_cap.retrieve()
        else:
            continue
        if not success:
            continue
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        face = mtcnn(frame, save_path = f'C:/ai/workspace/sh/newtrain/{label}/{num}/{fileName}{count}.jpg') #사진저장경로
        count=count+1

    v_cap.release()

- 사람 폴더별로 폴더 안 모든 영상에 대해 chimac 함수를 실행시킨다

In [ ]:
for path in range(len(real_list)):
    real_vid = [real_list[path] + x for x in os.listdir(real_list[path])]
    for vid in real_vid:
        chimac(vid, path, 'REAL')

In [ ]:
for path in range(len(fake_list)):
    fake_vid = [fake_list[path] + x for x in os.listdir(fake_list[path])]
    for vid in fake_vid:
        chimac(vid, path, 'FAKE')